# Quantitative Momentum Strategy

# Setup

In [1]:
# Numerical computing.
import numpy as np

# Tabular data manipulation.
import pandas as pd

# Requests.
import requests

# Statistics.
from scipy import stats

import xlsxwriter

import math

In [2]:
# API key.
from APIKey import APIKey

# Import List of Stocks

In [3]:
# sp_500_stocks.csv -- list of companies in S&P 500 (ticker symbol).
stocks = pd.read_csv('../sp_500_stocks.csv')

stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 1 columns):
Ticker    505 non-null object
dtypes: object(1)
memory usage: 4.1+ KB


In [4]:
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


# API

## Structure of an API Call

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={APIKey}'
data = requests.get(api_url).json()

data

{'companyName': 'Apple Inc',
 'marketcap': 2425788386618,
 'week52high': 146.45,
 'week52low': 92.59,
 'week52highSplitAdjustOnly': 145.54,
 'week52lowSplitAdjustOnly': 92.98,
 'week52change': 0,
 'sharesOutstanding': 17177232621,
 'float': 0,
 'avg10Volume': 67870552,
 'avg30Volume': 74167455,
 'day200MovingAvg': 129.95,
 'day50MovingAvg': 131.72,
 'employees': 150367,
 'ttmEPS': 4.52,
 'ttmDividendRate': 0.8432949002482356,
 'dividendYield': 0.006091602558314287,
 'nextDividendDate': '',
 'exDividendDate': '2021-04-24',
 'nextEarningsDate': '2021-07-13',
 'peRatio': 31.529271697169367,
 'beta': 1.6084669366036008,
 'maxChangePercent': 52.81306146435541,
 'year5ChangePercent': 5.315046487970219,
 'year2ChangePercent': 1.8644894615237535,
 'year1ChangePercent': 0.5723156753712766,
 'ytdChangePercent': 0,
 'month6ChangePercent': 0.06018189226319918,
 'month3ChangePercent': 0.1412530132280266,
 'month1ChangePercent': 0.1345445297074106,
 'day30ChangePercent': 0.13749862205059302,
 'day5C

## Parsing Retrieved Data

In [6]:
data['year1ChangePercent']

0.5723156753712766

# Simple Momentum Strategy

## Get the Required Data

### Execute Batch API Calls & Build a DataFrame

In [7]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]  

In [8]:
symbol_groups = list(chunks(stocks['Ticker'], 100))

symbol_groups

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

In [9]:
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [10]:
# Column names of required data.
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [11]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={APIKey}'
    
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 4 columns):
Ticker                     505 non-null object
Price                      505 non-null float64
One-Year Price Return      501 non-null object
Number of Shares to Buy    505 non-null object
dtypes: float64(1), object(3)
memory usage: 15.9+ KB


In [12]:
final_dataframe.head()

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,149.35,0.722395,N/A
1,AAL,22.39,0.745529,N/A
2,AAP,218.79,0.512824,N/A
3,AAPL,141.72,0.549805,N/A
4,ABBV,117.47,0.230984,N/A


## Remove Low-Momentum Stocks

In [13]:
# Sort (descending) by price return. 
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)

final_dataframe = final_dataframe[:50]

final_dataframe.reset_index(drop = True, inplace = True)

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,76.66,4.05825,N/A
1,FCX,37.54,2.30402,N/A
2,TPR,43.62,2.29742,N/A
3,DVN,30.11,1.82236,N/A
4,GPS,34.00,1.81052,N/A
5,SIVB,592.95,1.76976,N/A
6,KSS,56.43,1.7222,N/A
7,IVZ,26.64,1.65547,N/A
8,COF,158.60,1.63934,N/A
9,MGM,44.00,1.61891,N/A


## Number of Shares to Buy

In [14]:
# Portfolio size = 1 million US$.
portfolio_size = 1000000

In [15]:
# Top 50 stocks are equally weighted.
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,76.66,4.05825,260
1,FCX,37.54,2.30402,532
2,TPR,43.62,2.29742,458
3,DVN,30.11,1.82236,664
4,GPS,34.00,1.81052,588
5,SIVB,592.95,1.76976,33
6,KSS,56.43,1.7222,354
7,IVZ,26.64,1.65547,750
8,COF,158.60,1.63934,126
9,MGM,44.00,1.61891,454


In [16]:
# or
# Weight each stock by Market Capatilization or some other marketing strategy such as 1YearPriceReturn/(MarketCap/TotalMarket)
# total_market_cap = final_dataframe['Market Capitalization'].sum()
# for i in range(0, len(final_dataframe['Ticker'])):
#     final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor((float(portfolio_size) / final_dataframe['Price'][i])*(final_dataframe['Market Capitalization'][i]/total_market_cap))
# final_dataframe

## Export Recommended Trades

In [17]:
# Initializing XlsxWriter Object.
writer = pd.ExcelWriter('../recommended_trades/momentum_strategy_1.xlsx', engine='xlsxwriter')

final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [18]:
# Column formats.
# Background color.
background_color = '#0a0a23'

# Font color.
font_color = '#ffffff'

# String.
string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Price.
dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Integer.
integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Percent.
percent_format = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [19]:
# Apply column formats.
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['One-Year Price Return', percent_format],
                    'D': ['Number of Shares to Buy', integer_format],
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [20]:
# Save the excel file.
writer.save()

# Better Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future.

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

The abbreviation `hqm` stands for `high-quality momentum`.

## Get the Required Data

In [21]:
# Column names of required data.
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={APIKey}'
    
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 12 columns):
Ticker                           505 non-null object
Price                            505 non-null float64
Number of Shares to Buy          505 non-null object
One-Year Price Return            501 non-null object
One-Year Return Percentile       505 non-null object
Six-Month Price Return           501 non-null object
Six-Month Return Percentile      505 non-null object
Three-Month Price Return         501 non-null object
Three-Month Return Percentile    505 non-null object
One-Month Price Return           501 non-null object
One-Month Return Percentile      505 non-null object
HQM Score                        505 non-null object
dtypes: float64(1), object(11)
memory usage: 47.5+ KB


In [22]:
hqm_dataframe.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,152.40,N/A,0.69965,N/A,0.270951,N/A,0.169724,N/A,0.0963883,N/A,N/A
1,AAL,21.84,N/A,0.725492,N/A,0.367624,N/A,-0.101175,N/A,-0.140765,N/A,N/A
2,AAP,214.35,N/A,0.52507,N/A,0.362484,N/A,0.143827,N/A,0.111911,N/A,N/A
3,AAPL,142.05,N/A,0.558539,N/A,0.0588097,N/A,0.144095,N/A,0.13635,N/A,N/A
4,ABBV,117.26,N/A,0.228077,N/A,0.103829,N/A,0.0769638,N/A,0.026616,N/A,N/A


## Momentum Percentiles

In [23]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

# Handling Missing Values.
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        
        if type(hqm_dataframe.loc[row, change_col]) != float:
            hqm_dataframe.loc[row, change_col] = 0

# Computing various percentiles.
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
hqm_dataframe.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,152.40,N/A,0.69965,0.732673,0.270951,0.69505,0.169724,0.857426,0.0963883,0.891089,N/A
1,AAL,21.84,N/A,0.725492,0.758416,0.367624,0.847525,-0.101175,0.0356436,-0.140765,0.00594059,N/A
2,AAP,214.35,N/A,0.52507,0.558416,0.362484,0.841584,0.143827,0.786139,0.111911,0.914851,N/A
3,AAPL,142.05,N/A,0.558539,0.594059,0.0588097,0.207921,0.144095,0.788119,0.13635,0.946535,N/A
4,ABBV,117.26,N/A,0.228077,0.227723,0.103829,0.314851,0.0769638,0.576238,0.026616,0.677228,N/A


## HQM Score

`High-Quality Momentum (HQM) Score` will be the arithmetic mean of the 4 momentum percentile scores.

In [24]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
        
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,152.40,N/A,0.69965,0.732673,0.270951,0.69505,0.169724,0.857426,0.0963883,0.891089,0.794059
1,AAL,21.84,N/A,0.725492,0.758416,0.367624,0.847525,-0.101175,0.0356436,-0.140765,0.00594059,0.411881
2,AAP,214.35,N/A,0.52507,0.558416,0.362484,0.841584,0.143827,0.786139,0.111911,0.914851,0.775248
3,AAPL,142.05,N/A,0.558539,0.594059,0.0588097,0.207921,0.144095,0.788119,0.13635,0.946535,0.634158
4,ABBV,117.26,N/A,0.228077,0.227723,0.103829,0.314851,0.0769638,0.576238,0.026616,0.677228,0.44901


## Remove Low-Momentum Stocks

In [25]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace = True)

hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,838.95,N/A,1.17452,0.934653,0.571696,0.966337,0.48602,1,0.214281,0.992079,0.973267
1,FANG,101.11,N/A,1.44566,0.970297,1.08529,0.99802,0.221138,0.934653,0.139861,0.948515,0.962871
2,LB,76.92,N/A,4.08268,1,1.01308,0.99604,0.203409,0.916832,0.124347,0.934653,0.961881
3,IT,258.35,N/A,1.12009,0.922772,0.600153,0.972277,0.355748,0.99604,0.0926329,0.879208,0.942574
4,FTNT,250.90,N/A,0.829633,0.821782,0.678998,0.980198,0.345649,0.992079,0.158516,0.966337,0.940099
5,WAT,361.11,N/A,1.01206,0.89901,0.44499,0.928713,0.249401,0.958416,0.127581,0.938614,0.931188
6,OXY,33.91,N/A,0.876047,0.841584,0.890949,0.994059,0.198027,0.910891,0.115221,0.920792,0.916832
7,TGT,258.17,N/A,1.09911,0.918812,0.420089,0.89901,0.239721,0.950495,0.0763701,0.855446,0.905941
8,IDXX,659.60,N/A,0.958013,0.875248,0.301623,0.758416,0.346923,0.994059,0.182629,0.980198,0.90198
9,EXR,173.23,N/A,0.802343,0.809901,0.474869,0.944554,0.246689,0.954455,0.0949177,0.883168,0.89802


## Number of Shares to Buy

In [26]:
# Portfolio size = 1 million US$.
portfolio_size = 1000000

In [27]:
# Top 50 stocks are equally weighted.
position_size = float(portfolio_size) / len(hqm_dataframe.index)

for i in range(0, len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,838.95,23,1.17452,0.934653,0.571696,0.966337,0.48602,1,0.214281,0.992079,0.973267
1,FANG,101.11,197,1.44566,0.970297,1.08529,0.99802,0.221138,0.934653,0.139861,0.948515,0.962871
2,LB,76.92,260,4.08268,1,1.01308,0.99604,0.203409,0.916832,0.124347,0.934653,0.961881
3,IT,258.35,77,1.12009,0.922772,0.600153,0.972277,0.355748,0.99604,0.0926329,0.879208,0.942574
4,FTNT,250.90,79,0.829633,0.821782,0.678998,0.980198,0.345649,0.992079,0.158516,0.966337,0.940099
5,WAT,361.11,55,1.01206,0.89901,0.44499,0.928713,0.249401,0.958416,0.127581,0.938614,0.931188
6,OXY,33.91,589,0.876047,0.841584,0.890949,0.994059,0.198027,0.910891,0.115221,0.920792,0.916832
7,TGT,258.17,77,1.09911,0.918812,0.420089,0.89901,0.239721,0.950495,0.0763701,0.855446,0.905941
8,IDXX,659.60,30,0.958013,0.875248,0.301623,0.758416,0.346923,0.994059,0.182629,0.980198,0.90198
9,EXR,173.23,115,0.802343,0.809901,0.474869,0.944554,0.246689,0.954455,0.0949177,0.883168,0.89802


## Export Recommended Trades

In [28]:
# Initializing XlsxWriter Object.
writer = pd.ExcelWriter('../recommended_trades/momentum_strategy_2.xlsx', engine='xlsxwriter')

hqm_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [29]:
# Column formats.
# Background color.
background_color = '#0a0a23'

# Font color.
font_color = '#ffffff'

# String.
string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Price.
dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Integer.
integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Percent.
percent_format = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [30]:
# Apply column formats.
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Number of Shares to Buy', integer_format],
                    'D': ['One-Year Price Return', percent_format],
                    'E': ['One-Year Return Percentile', percent_format],
                    'F': ['Six-Month Price Return', percent_format],
                    'G': ['Six-Month Return Percentile', percent_format],
                    'H': ['Three-Month Price Return', percent_format],
                    'I': ['Three-Month Return Percentile', percent_format],
                    'J': ['One-Month Price Return', percent_format],
                    'K': ['One-Month Return Percentile', percent_format],
                    'L': ['HQM Score', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [31]:
# Save the excel file.
writer.save()